# BMI / CS 771: Homework Assignment 1

This project will use Jupyter Notebook in combination with Python scripts, offering a convenient way for you to quickly and easily interact with the code. A notebook contains many blocks of code, each of which can be run independently. You can run a cell with ctrl+enter or shift+enter (to move to the next cell).

If there are any notes you wish to leave for the teaching team, you may leave them here.

## Setup

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from utils import load_image, save_image
from student_code import Compose, Scale, RandomHorizontalFlip, RandomColor, RandomRotate, RandomSizedCrop
%load_ext autoreload
%autoreload 2

image1 = load_image('../data/dog.bmp')
image2 = load_image('../data/bird.bmp')
image3 = load_image('../data/cat.bmp')
image4 = load_image('../data/plane.bmp')

# display the dog and cat images
plt.figure(figsize=(3,3)); plt.axis('off'); plt.imshow(image1);
plt.figure(figsize=(3,3)); plt.axis('off'); plt.imshow(image2);
plt.show()

## Create Image Transforms

In [ ]:
# create an empty list and add transforms one by one
transforms = []
transforms.append(Scale(320))
transforms.append(RandomHorizontalFlip())
transforms.append(RandomColor(0.15))
transforms.append(RandomRotate(30))
transforms.append(RandomSizedCrop(224))
comp_transforms = Compose(transforms)
print(comp_transforms)


## Apply Transforms to Images
All functions you need to implement in this project can be found in student_code.py. You will need to fill in the missing code pieces in the Scale, RandomRotate and RandomColor.

In [ ]:
transformed_image1 = comp_transforms(image1)

# let's take a look at the results!
plt.figure(figsize=(4,4)); plt.axis('off'); plt.imshow(image1);
plt.figure(figsize=(4,4)); plt.axis('off'); plt.imshow(transformed_image1);

plt.show()


## Create More Results

In [ ]:
vis_img_list = []
for img in [image1, image2, image3, image4]:
    img_list = []
    for idx in range(10):
        img_list.append(comp_transforms(img))
    vis_img = np.concatenate(img_list, axis=1)
    vis_img_list.append(vis_img)

for vis_img in vis_img_list:
    plt.figure(figsize=(16,16)); plt.axis('off'); plt.imshow(vis_img);
    
plt.show()

## Save Results

In [ ]:
for file_idx, vis_img in zip(range(len(vis_img_list)), vis_img_list):
    save_image('../results/outputs_{:d}.jpg'.format(file_idx), vis_img)

## Data Augmentation and Input Pipeline

In [ ]:
from student_code import ToTensor, SimpleDataset, CenterCrop, Resize, Net
import torch
import torchvision

# now we will do some real training on cats and dogs

# set up the transforms (you can play around with the combination here)
train_transforms = []
train_transforms.append(Scale(72))
train_transforms.append(RandomHorizontalFlip())
train_transforms.append(RandomSizedCrop(64, area_range=(0.64, 1.0)))
train_transforms.append(ToTensor())
train_transforms = Compose(train_transforms)

test_transforms = []
test_transforms.append(Scale(72))
test_transforms.append(CenterCrop(64))
test_transforms.append(ToTensor())
test_transforms = Compose(test_transforms)
print("Image transforms used for training")
print(train_transforms)
print("Image transforms used for validation:")
print(test_transforms)

# let us try a toy dataset
batch_size = 16
train_dataset = SimpleDataset('../data/mini_train', file_ext='jpg', transforms=train_transforms)
val_dataset = SimpleDataset('../data/mini_val', file_ext='jpg', transforms=test_transforms)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# get some random training images
dataiter = iter(train_data_loader)
images, labels = next(dataiter)

# show images
nrow = batch_size // 2
vis_img = torchvision.utils.make_grid(images, nrow=nrow).numpy()
plt.figure(); plt.axis('off'); plt.imshow(np.transpose(vis_img, (1, 2, 0)))
plt.show()
classes = ("dog", "cat")
# print labels
output = ''
for i, j in enumerate(range(batch_size), 1):
    output += f'{classes[labels[j]]:5s}' + ['\t', '\n'][i % nrow == 0]
print(output)
del dataiter


In [ ]:
# network / loss function for training
net = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001)

# training loop
num_epochs = 6
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
# performance on the validation set
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
net.eval()
with torch.no_grad():
    for data in val_data_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the validation set: {100 * correct // total} %')